In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import random

In [ ]:
class DataLoader:
    def __init__(self,
                 datapath,
                 shuffle=True):
        self.shuffle = shuffle
        self.data = {}

        for root, _, files in os.walk(datapath):
            for name in files:
                filename = name.split('.')
                self.data[filename[0] + filename[1]] = np.load(os.path.join(root, name))

    def __call__(self):
        keys = list(self.data.keys())
        if self.shuffle:
            random.shuffle(keys)
        # итерация по ключам в словаре self.data
        for key in keys:
            # если маска целевого значения пустая, то пропускаем пример
            if np.all(self.data[key][:,:,-1,:,:] == 0.):
                continue
            seq = self.__getSequence(key)
            arrays = []
            badCount = 0
            # итерация по историческим данным
            for item in seq:
                # некоторые исторические данные могут отсутствовать
                try:
                    if np.all(self.data[item][:,:,-1,:,:] == 0.):
                        badCount += 1
                    arrays.append(self.data[item])
                except KeyError:
                    # print(f'No key: {item}')
                    badCount += 1
                    arrays.append(np.zeros_like(self.data[key]))
            # если пропусков в данных больше чем 30%, то пропускаем пример
            # print(f'Bad count: {badCount}')
            if badCount / len(arrays) > 0.3:
                continue
            else:
                x = np.concatenate(arrays, axis=1)
                y = self.data[key]
                for beam in range(16):
                    yield x[:,:,:,beam,0], y[:,:,:-1,beam,0]
    
    def __getSequence(self, key):
        keyDT = datetime.strptime(key, '%Y%m%d%H%M')
        # список массивов периодов за неделю до целевого массива
        weekBefore = []
        for i in range(24*7, 0, -2):
            hoursBefore = (keyDT-timedelta(hours=i)).strftime('%Y%m%d%H%M')
            weekBefore.append(hoursBefore)
        return weekBefore

In [ ]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
# генератор
def get_generator():

    rand_inp = tf.keras.layers.Input(shape=(70, 60, 7))

    inp = tf.keras.layers.Input(shape=(70, 5040, 7))

    rand_concat = tf.keras.layers.Concatenate(axis=2)([inp, rand_inp])

    reshape = tf.keras.layers.Reshape(target_shape=(70, 5100//60, 60, 7))(rand_concat)

    permute = tf.keras.layers.Permute((2,1,3,4))(reshape)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(permute)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(5, 5),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(7, 8, 8)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(28, 10, 10)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(52, 14, 14)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3DTranspose(
        filters=6, kernel_size=(1, 36, 36), activation="linear"
    )(x)

    x = tf.keras.layers.Reshape(target_shape=(70, 60, 6))(x)

    model = tf.keras.models.Model([inp, rand_inp], x)

    return model

# дискриминатор
def get_discriminator():
    hist_inp = tf.keras.layers.Input(shape=(70, 5040, 7))
    hist_x = tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 60))(hist_inp)
    hist_x = tf.keras.layers.BatchNormalization()(hist_x)
    hist_x = tf.keras.layers.LeakyReLU(0.3)(hist_x)

    hist_x = tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 60*12), strides=(2,2))(hist_x)
    hist_x = tf.keras.layers.BatchNormalization()(hist_x)
    hist_x = tf.keras.layers.LeakyReLU(0.3)(hist_x)

    # model = keras.models.Model(hist_inp, hist_x, name='discriminator')
    # return model

    gen_out_inp = tf.keras.layers.Input(shape=(70, 60, 6))
    gen_out_x = tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5))(gen_out_inp)
    gen_out_x = tf.keras.layers.BatchNormalization()(gen_out_x)
    gen_out_x = tf.keras.layers.LeakyReLU(0.3)(gen_out_x)

    gen_out_x = tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(2,2))(gen_out_x)
    gen_out_x = tf.keras.layers.BatchNormalization()(gen_out_x)
    gen_out_x = tf.keras.layers.LeakyReLU(0.3)(gen_out_x)

    # model = keras.models.Model(gen_out_inp, gen_out_x)
    # return model

    joined = tf.keras.layers.Concatenate(axis=2)([hist_x, gen_out_x])
    joined = tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 5))(joined)
    joined = tf.keras.layers.BatchNormalization()(joined)
    joined = tf.keras.layers.LeakyReLU(0.3)(joined)

    joined = tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 5), strides=(1,2))(joined)
    joined = tf.keras.layers.BatchNormalization()(joined)
    joined = tf.keras.layers.LeakyReLU(0.3)(joined)

    joined = tf.keras.layers.Conv2D(filters=6, kernel_size=(1, 5), strides=(1,2))(joined)
    joined = tf.keras.layers.BatchNormalization()(joined)
    joined = tf.keras.layers.LeakyReLU(0.3)(joined)

    joined = tf.keras.layers.Conv2D(filters=6, kernel_size=(1, 5),strides=(1,2))(joined)
    joined = tf.keras.layers.BatchNormalization()(joined)
    joined = tf.keras.layers.LeakyReLU(0.3)(joined)

    joined = tf.keras.layers.Conv2D(filters=6, kernel_size=(1, 5),strides=(1,2))(joined)
    joined = tf.keras.layers.BatchNormalization()(joined)
    joined = tf.keras.layers.LeakyReLU(0.3)(joined)

    joined = tf.keras.layers.Flatten()(joined)
    joined = tf.keras.layers.Dense(60, activation='sigmoid')(joined)
    
    model = tf.keras.models.Model([hist_inp, gen_out_inp], joined, name='discriminator')
    return model

In [ ]:
class RadarGAN(keras.Model):
    def __init__(self, discriminator, generator):
        super(RadarGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
        self.gen_mae_tracker = keras.metrics.Mean(name="generator_mae")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker, self.gen_mae_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(RadarGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.mae = keras.losses.MeanAbsoluteError()

    def train_step(self, data):

        # класс 1 - настоящие данные, класс 0 - дискриминированные данные
        x, y = data

        y_shape = tf.shape(y)

        # обучение дискриминатора
        for step in range(1):
            with tf.GradientTape() as tape:

                # выход генератора
                generated = tf.cast(self.generator([x, tf.random.uniform(shape=(y_shape[0], y_shape[1], y_shape[2], 7))]), dtype=tf.dtypes.float64)
                
                real = self.discriminator([x, y])
                fake = self.discriminator([x, generated])
                
                # перемешивание
                real_loss = self.loss_fn(tf.ones_like(real), real)
                fake_loss = self.loss_fn(tf.zeros_like(fake), fake)
                
                real_mixed = tf.math.round(tf.random.uniform(shape=(y_shape[0], y_shape[2]), minval=0, maxval=1, dtype=tf.dtypes.float64))

                # маски для перемешивания данных на входе дискриминатора
                real_mask_mixed = tf.reshape(real_mixed, shape=(y_shape[0], 1, y_shape[2], 1))
                real_mask_mixed = tf.repeat(real_mask_mixed, repeats=y_shape[1], axis=1)
                real_mask_mixed = tf.repeat(real_mask_mixed, repeats=y_shape[3], axis=3)
                fake_mask_mixed = tf.math.subtract(tf.ones(shape=tf.shape(real_mask_mixed), dtype=tf.dtypes.float64), real_mask_mixed)

                # перемешивание данных
                mixed = tf.math.add(tf.math.multiply(real_mask_mixed, y), tf.math.multiply(fake_mask_mixed, generated))

                predictions = self.discriminator([x, mixed])

                mixed_loss = self.loss_fn(real, predictions)

                d_loss = real_loss + fake_loss + mixed_loss


            grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(
                zip(grads, self.discriminator.trainable_weights)
            )
        
        # таргет генератора как выход дискриминатора
        misleading_labels = tf.ones(shape=tf.shape(real), dtype=tf.dtypes.float64)

        for step in range(1):
            # тренировка генератора
            with tf.GradientTape() as tape:
                fake_forecast = self.generator([x, tf.random.uniform(shape=(y_shape[0], y_shape[1], y_shape[2], 7))])
                predictions = self.discriminator([x, fake_forecast])
                g_loss = self.loss_fn(misleading_labels, predictions)
                g_mae = self.mae(y, fake_forecast)
            grads = tape.gradient(g_loss, self.generator.trainable_weights)
            self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        self.gen_mae_tracker.update_state(g_mae)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
            "g_mae": self.gen_mae_tracker.result()
            }

In [ ]:
# если обучать с нуля
radar_gan = RadarGAN(discriminator=get_discriminator(), generator=get_generator())

# дообучение
# radar_gan = RadarGAN(discriminator=keras.models.load_model('drive/MyDrive/radargan_discriminator.hdf5'), generator=keras.models.load_model('drive/MyDrive/radargan_generator.hdf5'))


radar_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss_fn=keras.losses.BinaryCrossentropy()
)

In [ ]:
batch_size = 1

train_loader = DataLoader('drive/MyDrive/2002-val')
# val_loader = DataLoader('drive/MyDrive/2002-val')

train_dataset = tf.data.Dataset.from_generator(train_loader,
                                         output_types=(tf.float64, tf.float64)).batch(batch_size)
# val_dataset = tf.data.Dataset.from_generator(val_loader,
#                                          output_types=(tf.float64, tf.float64)).batch(batch_size)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(train_loader,
                                         output_types=(tf.float64, tf.float64)).batch(1)

hist = radar_gan.fit(train_dataset, epochs=2)

In [ ]:
radar_gan.generator.save('drive/MyDrive/radargan_generator.hdf5')
radar_gan.discriminator.save('drive/MyDrive/radargan_discriminator.hdf5')

In [ ]:
import matplotlib.pyplot as plt
for x, y in train_dataset:
    true = y
    pred = radar_gan.generator.predict(x)
    break
fig, axs = plt.subplots(2)
axs[0].imshow(true[0, :, :, -1])
axs[0].set_title('True')
axs[1].imshow(pred[0, :, :, -1])
axs[1].set_title('Predicted')